In [1]:
import numpy as np
import cv2
import time


In [2]:
# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]


In [7]:
# Draw the predicted bounding box
def drawPred(classId, conf, left, top, right, bottom):
    # Draw a bounding box.
    cv2.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)

    label = '%.2f' % conf

    # Get the label for the class name and its confidence
    if classes:
        assert (classId < len(classes))
        label = '%s:%s' % (classes[classId], label)

    # Display the label at the top of the bounding box
    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(frame, (left, top - round(1.5 * labelSize[1])), (left + round(1.5 * labelSize[0]), top + baseLine),
                  (255, 255, 255), cv2.FILLED)
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 0), 1)




In [ ]:

# Remove the bounding boxes with low confidence using non-maxima suppression
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            if detection[4] > objectnessThreshold:
                scores = detection[5:]
                classId = np.argmax(scores)
                confidence = scores[classId]
                if confidence > confThreshold and classId == 32:
                    center_x = int(detection[0] * frameWidth)
                    center_y = int(detection[1] * frameHeight)
                    width = int(detection[2] * frameWidth)
                    height = int(detection[3] * frameHeight)
                    left = int(center_x - width / 2)
                    top = int(center_y - height / 2)
                    classIds.append(classId)
                    confidences.append(float(confidence))
                    boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(classIds[i], confidences[i], left, top, left + width, top + height)
    return boxes





# START OF MAIN PROGRAM
# Initialize the parameters
objectnessThreshold = 0.5 # Objectness threshold
confThreshold = 0.5       # Confidence threshold
nmsThreshold = 0.4        # Non-maximum suppression threshold
inpWidth = 416            # Width of network's input image
inpHeight = 416           # Height of network's input image

classesFiles = "coco.names"
classes = None
with open(classesFiles, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

modelConfig = 'yolov3.cfg'
modelWeights = 'yolov3.weights'

# Read model
net = cv2.dnn.readNetFromDarknet(cfgFile=modelConfig, darknetModel=modelWeights)
cap = cv2.VideoCapture('soccer-ball.mp4')
startTime = time.time()
startDetectionTime = time.time()
detectionTime = 0
tracker = cv2.TrackerKCF_create()
trackerInit = True
allowDetect = True
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        # Detect Object
        if allowDetect:
            print(detectionTime)
            blob = cv2.dnn.blobFromImage(frame, 1 / 255, (inpWidth, inpHeight), [0, 0, 0], 1, crop=False)
            net.setInput(blob)
            outs = net.forward(getOutputsNames(net))
            box = postprocess(frame, outs)
            if any(x is not 0 for x in box) and len(box) == 1:
                trackerBox = tuple(sum(box, []))
            # print("DetectorBox: " , box)
            allowDetect = False
            tracker = cv2.TrackerKCF_create()
            ok = tracker.init(frame, trackerBox)
            cv2.putText(frame, "Detecting Object!", (20, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)
            detectionTime = time.time() - startDetectionTime
            # print(detectionTime)

        # Initialise Tracker
        if trackerInit:
            ok = tracker.init(frame, (568, 323, 108, 102))
            trackerInit = False

        # print("trackerBox: " , trackerBox)
        if ok:
            # Tracking success
            p1 = (int(trackerBox[0]), int(trackerBox[1]))
            p2 = (int(trackerBox[0] + trackerBox[2]), int(trackerBox[1] + trackerBox[3]))
            cv2.rectangle(frame, p1, p2, (0, 0, 255), 2, 1)
            startDetectionTime = time.time()
            ok, trackerBox = tracker.update(frame)
        else:
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (20, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2)
            allowDetect = True
            # Detect for a maximum of 1.5 seconds
            if detectionTime > 1.5:
                allowDetect = False
                detectionTime = time.time() - startDetectionTime
                # Pause detection for 1.5 second
                if detectionTime > 3:
                    startDetectionTime = time.time()
                    detectionTime = 0
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()


<>:80: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:80: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-10-b96c943a6387>:80: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if any(x is not 0 for x in box) and len(box) == 1:


0
0.5994079113006592
1.2119221687316895
0
0.5018026828765869
0.9401838779449463
1.3991541862487793
0
0.48029327392578125
0.9223136901855469
1.3942182064056396
0
0.5315651893615723
1.0187568664550781
0
0.4718132019042969
0.9206492900848389
1.3905794620513916
0
0.5676121711730957
1.041147232055664
0
0.5215151309967041
0.988938570022583
1.4420294761657715
0
0.5037407875061035
0.9945206642150879
1.4611995220184326
0
0.527238130569458
0.9853987693786621
1.4467310905456543
0
0.5062210559844971
0.9622592926025391
1.443718671798706
0
0.5209946632385254
1.0693652629852295
0
0.5501608848571777
1.0692999362945557
0
0.5183377265930176
1.0005528926849365
0
0.5287125110626221
1.024960994720459
0
0.5869050025939941
1.0709905624389648
0
0.5143697261810303
1.015094518661499
